# Federated Protein Embeddings and Task Model Fitting with BioNeMo

This example notebook shows how to obtain protein learned representations in the form of embeddings using the ESM-1nv pre-trained model. The model is trained with NVIDIA's BioNeMo framework for Large Language Model training and inference. For more details, please visit NVIDIA BioNeMo Service at https://www.nvidia.com/en-us/gpu-cloud/bionemo.

This notebook will walk you through the task fitting workflow in the following sections:

* 
*
*

### Install requirements
Let's start by installing and importing library dependencies. We'll use requests to interact with the BioNeMo service, BioPython to parse FASTA sequences into SeqRecord objects, scikit-learn for classification tasks, and matplotlib for visualization.

In [1]:
!pip install -r requirements.txt

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
You should consider upgrading via the '/usr/bin/python -m pip install --upgrade pip' command.


### Obtaining the protein embeddings using the BioNeMo ESM-1nv model
Using BioNeMo, users can obtain numerical vector representations of protein sequences called embeddings. Protein embeddings can then be used for visualization or making downstream predictions.

Here we are interested in training a neural network to predict subcellular location from an embedding.

The data we will be using comes from the paper [Light attention predicts protein location from the language of life](https://academic.oup.com/bioinformaticsadvances/article/1/1/vbab035/6432029) by Stärk et al. In this paper, the authors developed a machine learning algorithm to predict the subcellular location of proteins from sequence through protein langage models that are similar to those hosted by BioNeMo. Protein subcellular location refers to where the protein localizes in the cell, for example a protein my be expressed in the Nucleus or in the Cytoplasm. Knowing where proteins localize can provide insights into the underlying mechanisms of cellular processes and help identify potential targets for drug development. The following image includes a few examples of subcellular locations in an animal cell:


(Image freely available at https://pixabay.com/images/id-48542)

### Dataset sourcing
For our target input sequences, we will point to FASTA sequences in a benchmark dataset called Fitness Landscape Inference for Proteins (FLIP). FLIP encompasses experimental data across adeno-associated virus stability for gene therapy, protein domain B1 stability and immunoglobulin binding, and thermostability from multiple protein families.

In [2]:
# Example protein dataset location
fasta_url= "http://data.bioembeddings.com/public/FLIP/fasta/scl/mixed_soft.fasta"

First, we define the source of example protein dataset with the FASTA sequences. This data follows the [biotrainer](https://github.com/sacdallago/biotrainer/blob/main/docs/data_standardization.md) standard, so it includes information about the class in the FASTA header, and the protein sequence. Here are two example sequences in this file:

```
>Sequence1 TARGET=Cell_membrane SET=train VALIDATION=False
MMKTLSSGNCTLNVPAKNSYRMVVLGASRVGKSSIVSRFLNGRFEDQYTPTIEDFHRKVYNIHGDMYQLDILDTSGNHPFPAM
RRLSILTGDVFILVFSLDSRESFDEVKRLQKQILEVKSCLKNKTKEAAELPMVICGNKNDHSELCRQVPAMEAELLVSGDENC
AYFEVSAKKNTNVNEMFYVLFSMAKLPHEMSPALHHKISVQYGDAFHPRPFCMRRTKVAGAYGMVSPFARRPSVNSDLKYIKA
KVLREGQARERDKCSIQ
>Sequence4833 TARGET=Nucleus SET=train VALIDATION=False
MARTKQTARKSTGGKAPRKQLATKAARKSAPATGGVKKPHRFRPGTVALREIRKYQKSTELLIRKLPFQRLVREIAQDFKTDL
RFQSSAVAALQEAAEAYLVGLFEDTNLCAIHAKRVTIMPKDIQLARRIRGERA
Note the following attributes in the FASTA header:
```

* `TARGET` attribute holds the subcellular location classification for the sequence, for instance Cell_membrane and Nucleus. This dataset includes a total of ten subcellelular location classes -- more on that below.
* `SET` attribute defines whether the sequence should be used for training (train) or testing (test)
* `VALIDATION` attribute defines whether the sequence should be used for validation (all sequences where this is True are also in set=train)

### Downloading the protein sequences and subcellular location annotations
In this step we download the FASTA file defined above and parse the sequences into a list of BioPython SeqRecord objects.



In [3]:
import io
import requests
from Bio import SeqIO

# Download the FASTA file from FLIP: https://github.com/J-SNACKKB/FLIP/tree/main/splits/scl
fasta_content = requests.get(fasta_url, headers={
    'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x86)'
}).content.decode('utf-8')
fasta_stream = io.StringIO(fasta_content)

# Obtain a list of SeqRecords/proteins which contain sequence and attributes
# from the FASTA header
proteins = list(SeqIO.parse(fasta_stream, "fasta"))
print(f"Downloaded {len(proteins)} sequences")

Downloaded 13949 sequences


### Data splitting
Next, we prepare the data for simulating federated learning using `n_clients`.

In [4]:
n_clients = 3
# limiting to the proteins with sequence length<512 for embedding queries
MAX_SEQUENCE_LEN = 512
seed=0  # 0,1 works with 100
out_dir = "/tmp/fasta/mixed_soft"
split_alpha = 10.0

import os
import re
import numpy as np
import pandas as pd
import uuid

from importlib import reload
import split_data
reload(split_data)
from split_data import split, list_to_dataframe
np.random.seed(seed)

# Extract meta data and split
data = []
for i, x in enumerate(proteins):
        if len(str(x.seq)) > MAX_SEQUENCE_LEN:
            continue
            
        entry = {key: value for key, value in re.findall(r"([A-Z_]+)=(-?[A-z0-9]+[.0-9]*)", x.description)}
        entry["sequence"] = str(x.seq)
        entry["id"] = str(i)
       
        data.append(entry)
print(f"Read {len(data)} valid sequences.")
               
# Split the data and save for each client
# Note, test_data is kept the same on each client and is not split
split(proteins=data, num_sites=n_clients, split_dir=out_dir, alpha=split_alpha)

Read 8619 valid sequences.
Partition protein dataset with 10 classes into 3 sites with Dirichlet sampling under alpha 10.0
{'site-1': {'Cell_membrane': 114,
            'Cytoplasm': 375,
            'Endoplasmic_reticulum': 201,
            'Extracellular': 472,
            'Golgi_apparatus': 32,
            'Lysosome': 42,
            'Mitochondrion': 346,
            'Nucleus': 455,
            'Peroxisome': 26,
            'Plastid': 118},
 'site-2': {'Cell_membrane': 241,
            'Cytoplasm': 325,
            'Endoplasmic_reticulum': 164,
            'Extracellular': 481,
            'Golgi_apparatus': 92,
            'Lysosome': 62,
            'Mitochondrion': 390,
            'Nucleus': 680,
            'Peroxisome': 32,
            'Plastid': 114},
 'site-3': {'Cell_membrane': 159,
            'Cytoplasm': 463,
            'Endoplasmic_reticulum': 97,
            'Extracellular': 379,
            'Golgi_apparatus': 40,
            'Lysosome': 45,
            'Mitochondrion'

### Federated embedding extraction
Running inference of the ESM-1nv model to extract embeddings requires a GPU with at least 12 GB memory. Here we run inference on each client sequentially using one thread to preserve GPU memory.

In [5]:
from nvflare import SimulatorRunner    

simulator = SimulatorRunner(
    job_folder="jobs/embeddings",
    workspace="/tmp/nvflare/bionemo/embeddings",
    n_clients=n_clients,
    threads=1  # due to memory constraints, we run the client execution sequentially in one thread
)
run_status = simulator.run()
print("Simulator finished with run_status", run_status)

2023-08-10 16:39:07,271 - SimulatorRunner - INFO - Create the Simulator Server.
2023-08-10 16:39:07,279 - Cell - INFO - server: creating listener on tcp://0:44147
2023-08-10 16:39:07,312 - Cell - INFO - server: created backbone external listener for tcp://0:44147
2023-08-10 16:39:07,314 - ConnectorManager - INFO - 17239: Try start_listener Listener resources: {'secure': False, 'host': 'localhost'}
2023-08-10 16:39:07,316 - nvflare.fuel.f3.sfm.conn_manager - INFO - Connector [CH00002 PASSIVE tcp://0:40906] is starting
2023-08-10 16:39:07,819 - Cell - INFO - server: created backbone internal listener for tcp://localhost:40906
2023-08-10 16:39:07,822 - nvflare.fuel.f3.sfm.conn_manager - INFO - Connector [CH00001 PASSIVE tcp://0:44147] is starting
2023-08-10 16:39:07,913 - nvflare.fuel.hci.server.hci - INFO - Starting Admin Server localhost on Port 47411
2023-08-10 16:39:07,914 - SimulatorRunner - INFO - Deploy the Apps.
2023-08-10 16:39:08,696 - SimulatorRunner - INFO - Create the simulat

[NeMo W 2023-08-10 16:39:25 experimental:27] Module <class 'nemo.collections.nlp.models.text_normalization_as_tagging.thutmose_tagger.ThutmoseTaggerModel'> is experimental, not ready for production and is not fully supported. Use at your own risk.


2023-08-10 16:39:26,310 - ServerRunner - INFO - [identity=simulator_server, run=simulate_job, wf=bionemo_inference, peer=site-1, peer_run=simulate_job, task_name=bionemo_inference, task_id=4b32f2e2-9453-4b35-a47e-87d829c6066a]: assigned task to client site-1: name=bionemo_inference, id=4b32f2e2-9453-4b35-a47e-87d829c6066a
2023-08-10 16:39:26,313 - ServerRunner - INFO - [identity=simulator_server, run=simulate_job, wf=bionemo_inference, peer=site-1, peer_run=simulate_job, task_name=bionemo_inference, task_id=4b32f2e2-9453-4b35-a47e-87d829c6066a]: sent task assignment to client. client_name:site-1 task_id:4b32f2e2-9453-4b35-a47e-87d829c6066a


[NeMo W 2023-08-10 16:39:26 experimental:27] Module <class 'nemo.collections.asr.modules.audio_modules.SpectrogramToMultichannelFeatures'> is experimental, not ready for production and is not fully supported. Use at your own risk.


2023-08-10 16:39:26,315 - GetTaskCommand - INFO - return task to client.  client_name: site-1  task_name: bionemo_inference   task_id: 4b32f2e2-9453-4b35-a47e-87d829c6066a  sharable_header_task_id: 4b32f2e2-9453-4b35-a47e-87d829c6066a
[NeMo I 2023-08-10 16:39:26 utils:250] Restoring model from /tmp/nvflare/bionemo/embeddings/simulate_job/app_site-1/models/esm1nv.nemo
[NeMo I 2023-08-10 16:39:26 utils:254] Loading model class: bionemo.model.protein.esm1nv.esm1nv_model.ESM1nvModel
2023-08-10 16:39:26,426 - pytorch_lightning.utilities.rank_zero - INFO - GPU available: True (cuda), used: True
2023-08-10 16:39:26,427 - pytorch_lightning.utilities.rank_zero - INFO - TPU available: False, using: 0 TPU cores
2023-08-10 16:39:26,427 - pytorch_lightning.utilities.rank_zero - INFO - IPU available: False, using: 0 IPUs
2023-08-10 16:39:26,427 - pytorch_lightning.utilities.rank_zero - INFO - HPU available: False, using: 0 HPUs
[NeMo I 2023-08-10 16:39:26 exp_manager:374] Experiments will be logged 

...

Predicting DataLoader 0: 100%|██████████| 31/31 [00:38<00:00,  1.25s/it]2023-08-10 16:42:23,160 - ServerRunner - INFO - [identity=simulator_server, run=simulate_job, wf=bionemo_inference, peer=site-3, peer_run=simulate_job]: got result from client site-3 for task: name=bionemo_inference, id=9b6831ea-a32a-4a96-bfa3-49067326bc34
2023-08-10 16:42:23,163 - ServerRunner - INFO - [identity=simulator_server, run=simulate_job, wf=bionemo_inference, peer=site-3, peer_run=simulate_job, peer_rc=OK, task_name=bionemo_inference, task_id=9b6831ea-a32a-4a96-bfa3-49067326bc34]: finished processing client result by bionemo_inference
2023-08-10 16:42:23,165 - SubmitUpdateCommand - INFO - submit_update process. client_name:site-3   task_id:9b6831ea-a32a-4a96-bfa3-49067326bc34
2023-08-10 16:42:23,170 - SimulatorClientRunner - INFO - Simulate Run client: site-1 on GPU group: None
2023-08-10 16:42:23,209 - BioNeMoInference - INFO - [identity=simulator_server, run=simulate_job, wf=bionemo_inference]: task bi

[NeMo W 2023-08-10 16:42:35 experimental:27] Module <class 'nemo.collections.nlp.models.text_normalization_as_tagging.thutmose_tagger.ThutmoseTaggerModel'> is experimental, not ready for production and is not fully supported. Use at your own risk.


2023-08-10 16:42:36,330 - ServerRunner - INFO - [identity=simulator_server, run=simulate_job, wf=bionemo_inference, peer=site-1, peer_run=simulate_job]: server runner is finalizing - asked client to end the run
2023-08-10 16:42:36,332 - GetTaskCommand - INFO - return task to client.  client_name: site-1  task_name: __end_run__   task_id:   sharable_header_task_id: 
2023-08-10 16:42:36,338 - SimulatorClientRunner - INFO - Simulate Run client: site-2 on GPU group: None
2023-08-10 16:42:36,341 - FederatedClient - INFO - Shutting down client run: site-1
2023-08-10 16:42:36,343 - FederatedClient - INFO - Shutting down client run: site-2
2023-08-10 16:42:36,344 - FederatedClient - INFO - Shutting down client run: site-3
2023-08-10 16:42:36,345 - ServerRunner - INFO - [identity=simulator_server, run=simulate_job, wf=bionemo_inference]: asked to abort - triggered abort_signal to stop the RUN


[NeMo W 2023-08-10 16:42:36 experimental:27] Module <class 'nemo.collections.asr.modules.audio_modules.SpectrogramToMultichannelFeatures'> is experimental, not ready for production and is not fully supported. Use at your own risk.


2023-08-10 16:42:40,109 - MPM - INFO - MPM: Good Bye!
Simulator finished with run_status 0


### Inspecting the embeddings and labels
Embeddings returned from the BioNeMo server are vectors of fixed size for each input sequence. In other words, if we input 10 sequences, we will obtain a matrix `10xD`, where `D` is the size of the embedding (in the case of ESM-1nv, `D=768`). At a glance, these real-valued vector embeddings don't show any obvious features (see the printout in the next cell). But these vectors do contain information that can be used in downstream models to reveal properties of the protein, for example the subcellular location as we'll explore below.

In [6]:
# load embeddings from site-1
import pickle
protein_embeddings = pickle.load(open(os.path.join(out_dir, "data_site-1.pkl"), "rb"))
print(f"Loaded {len(protein_embeddings)} embeddings from site-1.")

for i in range(4):
    protein_embedding = protein_embeddings[i]
    print(f"Inference result contains {list(protein_embedding.keys())}")
    x = protein_embedding["embeddings"]
    print(f"{protein_embedding['id']}: range {np.min(x)}-{np.max(x)}, mean={np.mean(x)}, shape={x.shape}")

Loaded 3863 embeddings from site-1.
Inference result contains ['embeddings', 'hiddens', 'sequence', 'id']
4375: range -0.86572265625-1.171875, mean=-0.00187550054397434, shape=(768,)
Inference result contains ['embeddings', 'hiddens', 'sequence', 'id']
3993: range -0.78369140625-1.2783203125, mean=-0.00043818666017614305, shape=(768,)
Inference result contains ['embeddings', 'hiddens', 'sequence', 'id']
9909: range -0.763671875-1.79296875, mean=-0.0019035814329981804, shape=(768,)
Inference result contains ['embeddings', 'hiddens', 'sequence', 'id']
3304: range -0.87451171875-1.5302734375, mean=-0.003111470490694046, shape=(768,)


Let's enumerate the labels corresponding to potential subcellular locations.

In [7]:
# Let's also print all the labels

labels = set([entry['TARGET'] for entry in data])

for i, label in enumerate(labels):
    print(f"{i+1}. {label.replace('_', ' ')}")

1. Cytoplasm
2. Plastid
3. Cell membrane
4. Endoplasmic reticulum
5. Extracellular
6. Nucleus
7. Golgi apparatus
8. Lysosome
9. Peroxisome
10. Mitochondrion


### Training a MLP to predict subcellular location
To be able to classify proteins for their subcellular location, we train a simple scikit-learn Multi-layer Perceptron (MPL) classifier. The MLP model uses a network of hidden layers to fit the input embedding vectors to the model classes (the cellular locations above). In the call below, we define the MLP to use the Adam optimizer with a network of 32 hidden layers, defining a random state (or seed) for reproducibility, and trained for a maximum of 500 iterations.

#### Local training

In [ ]:
os.environ["SIM_LOCAL"] = "True"

simulator = SimulatorRunner(
    job_folder="jobs/fedavg",
    workspace=f"/tmp/nvflare/bionemo/local_alpha{split_alpha}",
    n_clients=n_clients,
    threads=n_clients
)
run_status = simulator.run()
print("Simulator finished with run_status", run_status)

2023-08-10 16:42:41,984 - SimulatorRunner - INFO - Create the Simulator Server.
2023-08-10 16:42:41,989 - Cell - INFO - server: creating listener on tcp://0:48079
2023-08-10 16:42:42,008 - Cell - INFO - server: created backbone external listener for tcp://0:48079
2023-08-10 16:42:42,010 - ConnectorManager - INFO - 17907: Try start_listener Listener resources: {'secure': False, 'host': 'localhost'}
2023-08-10 16:42:42,012 - nvflare.fuel.f3.sfm.conn_manager - INFO - Connector [CH00002 PASSIVE tcp://0:47331] is starting
2023-08-10 16:42:42,514 - Cell - INFO - server: created backbone internal listener for tcp://localhost:47331
2023-08-10 16:42:42,518 - nvflare.fuel.f3.sfm.conn_manager - INFO - Connector [CH00001 PASSIVE tcp://0:48079] is starting
2023-08-10 16:42:42,609 - nvflare.fuel.hci.server.hci - INFO - Starting Admin Server localhost on Port 57919
2023-08-10 16:42:42,610 - SimulatorRunner - INFO - Deploy the Apps.
2023-08-10 16:42:42,617 - SimulatorRunner - INFO - Create the simulat

/usr/local/lib/python3.8/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:679: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1) reached and the optimization hasn't converged yet.
  warnings.warn(


2023-08-10 16:42:46,356 - BioNeMoMLPModelPersistor - INFO - [identity=simulator_server, run=simulate_job]: MLPClassifier coefficients [(768, 512), (512, 256), (256, 128), (128, 10)], intercepts [(512,), (256,), (128,), (10,)]
2023-08-10 16:42:46,358 - ServerRunner - INFO - [identity=simulator_server, run=simulate_job]: starting workflow scatter_gather_ctl (<class 'nvflare.app_common.workflows.scatter_and_gather.ScatterAndGather'>) ...
2023-08-10 16:42:46,360 - ScatterAndGather - INFO - [identity=simulator_server, run=simulate_job]: Initializing ScatterAndGather workflow.
2023-08-10 16:42:46,365 - ServerRunner - INFO - [identity=simulator_server, run=simulate_job]: Workflow scatter_gather_ctl (<class 'nvflare.app_common.workflows.scatter_and_gather.ScatterAndGather'>) started
2023-08-10 16:42:46,375 - ScatterAndGather - INFO - [identity=simulator_server, run=simulate_job, wf=scatter_gather_ctl]: Beginning ScatterAndGather training phase.
2023-08-10 16:42:46,377 - ScatterAndGather - INFO

#### Federated learning

In [ ]:
import os
os.environ["SIM_LOCAL"] = "False"

simulator = SimulatorRunner(
    job_folder="jobs/fedavg",
    workspace=f"/tmp/nvflare/bionemo/fedavg_alpha{split_alpha}",
    n_clients=n_clients,
    threads=n_clients
)
run_status = simulator.run()
print("Simulator finished with run_status", run_status)